In [ ]:
import numpy as np
from numba import jit
from itertools import combinations
import statistics
import ctypes
import time
import sys
import random
import math
import cProfile

class Gamer:
  __slots__ = ['former_round', 'output', 'points']
  def __init__(self):
    self.former_round = None
    self.output = None
    self.points = 0
  def end_of_turns(self):
    self.former_round, self.output = None, None
  def make_decision(self):
    raise NotImplementedError

In [ ]:
class copycat(Gamer):
  def __init__(self):
    super().__init__()
  def make_decision(self): #copycat copies the other's decision
    if self.former_round == None:
      self.output = 1
      return self.output
    else:
      self.output = self.former_round
      return self.output

class the_believer(Gamer):
  def __init__(self):
    super().__init__()
  def make_decision(self): #the_believer always trusts
    self.output = 1
    return self.output

class the_selfish(Gamer):
  def __init__(self):
    super().__init__()
  def make_decision(self): #the_selfish always betrays
    self.output = 0
    return self.output

class the_chaos(Gamer):
  def __init__(self):
    super().__init__()
  def make_decision(self): #the_choas returns random value
    self.output = random.randint(0, 1)
    return self.output

class the_stubborn(Gamer):
  __slots__ = ['is_betrayed']
  def __init__(self):
    super().__init__()
    self.is_betrayed = 0
  def make_decision(self): #trust until betrayed, and then always betray
    if self.former_round == None or self.former_round == 1 and self.is_betrayed == 0:
      self.output = 1
      return self.output
    self.is_betrayed = 1
    self.output = 0
    return self.output
  def end_of_turns(self):
    super().end_of_turns()
    self.is_betrayed = 0

class the_generous_copycat(Gamer):
  __slots__ = ['tolerance', 'betrayed_times']
  def __init__(self):
    super().__init__()
    self.tolerance = 2
    self.betrayed_times = 0
  def make_decision(self): #trust unless betrayed for consecutive tolerance times
    if self.former_round == 0:
      self.betrayed_times += 1
    else:
      self.betrayed_times = 0
    if self.former_round == None or self.betrayed_times < self.tolerance:
      self.output = 1
      return self.output
    self.output = 0
    return self.output
  def end_of_turns(self):
    super().end_of_turns()
    self.betrayed_times = 0

class pavlov(Gamer):
  __slots__ = ['default_output']
  def __init__(self):
    super().__init__()
    self.default_output =1
  def make_decision(self): #if former_round == output, cooperate, else defect
    if self.former_round == None:
      self.output = self.default_output
      return self.output
    if self.former_round == self.output:
      self.output = 1
      return self.output
    self.output = 0
    return self.output

class majority_follower(Gamer):
  __slots__ = ['oppo_total', 'default_output']
  def __init__(self):
    super().__init__()
    self.oppo_total = 0
    self.default_output = 1
  def make_decision(self):
    if self.former_round == 1:
      self.oppo_total += 1
    elif self.former_round == 0:
      self.oppo_total += -1
    if self.former_round == None or self.oppo_total == 0:
      self.output = self.default_output
      return self.output
    if self.oppo_total > 0:
      self.output = 1
      return self.output
    self.output = 0
    return self.output
  def end_of_turns(self):
    super().end_of_turns()
    self.oppo_total = 0

class the_mean_copycat(Gamer):
  __slots__ = ['revenge_times', 'revenge_count']
  def __init__(self):
    super().__init__()
    self.revenge_times = 2 #default
    self.revenge_count = 0
  def make_decision(self):
    if self.revenge_count > 0:
      self.output = 0
      self.revenge_count += 1
      if self.revenge_count == self.revenge_times:
        self.revenge_count =0
      return self.output
    if self.former_round == 0:
      self.output = 0
      self.revenge_count += 1
      return self.output
    self.output = 1
    return self.output
  def end_of_turns(self):
    super().end_of_turns()
    self.revenge_count = 0


class the_vicious_copycat(Gamer):
  def __init__(self):
    super().__init__()
  def make_decision(self):
    if self.former_round == None:
      self.output = 0
      return self.output
    else:
      self.output = self.former_round
      return self.output

class operator_gamer(Gamer):
  __slots__ = ['default_output' ,'operator']
  def __init__(self):
    super().__init__()
    self.default_output = 1
    self.operator = 10
    # self.operator default value is 10, namely a copycat
    # self.operator must be an integer in range(16), namely 0 to 15
  def make_decision(self):
    if self.operator > 15 or self.operator < 0:
      raise ValueError('operator must be an integer in range(16)')
    if self.former_round == None:
      self.output = self.default_output
      return self.output
    temp1 = self.output
    temp2 = self.former_round
    if temp1 and temp2:
      self.output = (self.operator >> 3) & 1
    elif not (temp1 or temp2):
      self.output = self.operator & 1
    elif temp1:
      self.output = (self.operator >> 2) & 1
    else:
      self.output = (self.operator >> 1) & 1
    return self.output

class the_tricker(Gamer):
  __slots__ = ['cooperate_times']
  def __init__(self):
    super().__init__()
    self.cooperate_times = 1
  def make_decision(self):
    if self.cooperate_times > 0:
      self.output = 1
      self.cooperate_times -= 1
      return self.output
    self.output = 0
    return self.output
  def end_of_turns(self):
    super().end_of_turns()
    self.cooperate_times = 1

class detective(Gamer):
  __slots__ = ['sequence_deci', 'is_betrayed', 'count_turns']
  def __init__(self):
    super().__init__()
    self.sequence_deci = 13 #default is 13, namely [1, 1, 0, 1]
    self.is_betrayed = 0
    self.count_turns = 0
  def make_decision(self):
    if len(format(self.sequence_deci, 'b')) > self.count_turns:
      self.output = self.sequence_deci>>(len(format(self.sequence_deci, 'b'))-self.count_turns-1) & 1
      self.count_turns += 1
      if self.former_round == 0:
        self.is_betrayed = 1
      return self.output
    if self.is_betrayed == 1:
      temp = copycat()
      temp.former_round, temp.output = self.former_round, self.output
      self.output = temp.make_decision()
      return self.output
    self.output = the_selfish().make_decision() #the_selfish is independent on former_round so no updating
    return self.output
  def end_of_turns(self):
    super().end_of_turns()
    self.count_turns = 0
    self.is_betrayed = 0

class remote_controller(Gamer):
  def __init__(self):
    super().__init__()
  def make_decision(self):
    print('former output:', self.output)
    print('former return:', self.former_round)
    self.output = int(input('Enter your decision: '))
    self.output = 0 if self.output == 0 else 1
    return self.output
  def end_of_turns(self):
    super().end_of_turns()

# The following five columns are the old game_type_list currently abandoned

In [ ]:
class type_list:
  def __init__(self, type):
    self.type = type
    self.list = []

In [ ]:
def get_pairs(listx):
  return [(i, j) for i in range(len(listx)) for j in range(i+1, len(listx))]
print(get_pairs([1, 2, 3, 4, 5]))

[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


In [ ]:
class game_type_list():
  def __init__(self):
    attributes_game = {'gamers': [], 'num_of_gamers': 0, 'num_of_turns': 10,
              'TT': [2,2], 'TB': [-1, 3], 'BB': [0, 0]}
    for name, value in attributes_game.items():
      setattr(self, name, value)
  # use list of lists to store gamers instead of a plain list
  def one_round(self, input1, input2):
    if input1 == input2 == 1: #(1, 1)
      return self.TT
    elif input1 == 1: #(1, 0)
      return self.TB
    elif input2 == 1: #(0, 1)
      return [self.TB[1], self.TB[0]]
    else: #(0, 0)
      return self.BB

  def add_gamer(self, gamer_type, n):
    for type_listx in self.gamers:
      if type(gamer_type()) == type_listx.type:
        for i in range(n):
          type_listx.list.append(gamer_type())
        self.num_of_gamers += n
        return
    self.gamers.append(type_list(type(gamer_type())))
    for i in range(n):
      self.gamers[-1].list.append(gamer_type())
    self.num_of_gamers += n

  def remove_gamer_a(self, gamer_type, n):
    for type_listx in self.gamers:
      if type_listx.type == type(gamer_type()):
        if n > len(type_listx.list):
          print('removals out of range')
          return
        else:
          del type_listx.list[-n:]
          self.num_of_gamers -= n
          return
    print('No such gamer')

  def run_matches(self):
    for type_listx in self.gamers:
      self.game_iteration(type_listx)
    pairs_x = get_pairs(self.gamers)
    for l in pairs_x:
      for gamer1 in self.gamers[l[0]].list:
        for gamer2 in self.gamers[l[1]].list:
          self.play_match(gamer1, gamer2)

  def play_match(self, gamer1, gamer2):
    for i in range(self.num_of_turns):
      outcome = self.one_round(gamer1.make_decision(), gamer2.make_decision())
      gamer1.points += outcome[0]
      gamer2.points += outcome[1]
      gamer1.former_round = gamer2.output
      gamer2.former_round = gamer1.output
    gamer1.end_of_turns()
    gamer2.end_of_turns()


  def game_iteration(self, type_listx):
    pairs = get_pairs(type_listx.list)
    for l in pairs:
      self.play_match(type_listx.list[l[0]], type_listx.list[l[1]])

In [ ]:
def old_test_case():
  game2 = game_type_list()
  game2.num_of_turns = 10
  game2.add_gamer(copycat, 100)
  game2.add_gamer(the_believer, 100)
  game2.add_gamer(the_selfish, 100)
  game2.add_gamer(pavlov, 100)
  game2.add_gamer(the_stubborn, 100)
  game2.add_gamer(the_generous_copycat, 100)
  game2.add_gamer(majority_follower, 100)
  game2.add_gamer(the_tricker, 100)
  game2.add_gamer(the_vicious_copycat, 0)
  print(game2.num_of_gamers)
  game2.run_matches()
  for i in game2.gamers:
    if len(i.list) > 0:
      print(i.type, ': ', i.list[0].points, end='\n')
  print('\n')
  game2.remove_gamer_a(the_selfish, 10)
  print(game2.num_of_gamers, len(game2.gamers[4].list))
  print(len(game2.gamers))
  for i in game2.gamers:
    print(i.type, ':', len(i.list), end='\n')
  print('\n')
  print(game2.gamers[0].type)

In [ ]:
old_test_case()

800
<class '__main__.copycat'> :  11980
<class '__main__.the_believer'> :  10280
<class '__main__.the_selfish'> :  6300
<class '__main__.pavlov'> :  11180
<class '__main__.the_stubborn'> :  11980
<class '__main__.the_generous_copycat'> :  11780
<class '__main__.majority_follower'> :  11880
<class '__main__.the_tricker'> :  7298


790 100
9
<class '__main__.copycat'> : 100
<class '__main__.the_believer'> : 100
<class '__main__.the_selfish'> : 90
<class '__main__.pavlov'> : 100
<class '__main__.the_stubborn'> : 100
<class '__main__.the_generous_copycat'> : 100
<class '__main__.majority_follower'> : 100
<class '__main__.the_tricker'> : 100
<class '__main__.the_vicious_copycat'> : 0


<class '__main__.copycat'>


# **The following are the new game() definition with a dctionary to store types and gamers**

In [ ]:
class IncoherenceError(Exception):
  def __init__(self, message):
    super().__init__(message)

In [ ]:
from collections import Counter
from collections import defaultdict

def round_half_up(n):
    return math.floor(n + 0.5)


#for those to be replaced, collect their gamer_type and list_index
#for those to be duplicated. collect their gamer_type and nums

def get_replacement(tuples, t):
  sorted_tuples = sorted(tuples, key=lambda x: x[2])
  top_border_value = sorted_tuples[-t][2]
  bottom_border_value = sorted_tuples[t-1][2]

  top_tied_tuples = [i for i in sorted_tuples if i[2] == top_border_value]
  bottom_tied_tuples = [j for j in sorted_tuples if j[2] == bottom_border_value]
  temp_a = len(bottom_tied_tuples)
  temp_b = len(top_tied_tuples)

  to_be_replaced = [i for i in sorted_tuples if i[2] < bottom_border_value]
  to_be_duplicated = [j for j in sorted_tuples if j[2] > top_border_value]
  # count to_be_duplicated but not to_be_replaced
  to_be_duplicated_count = Counter(i[0] for i in to_be_duplicated)


  bottom_tied_counts = list(Counter(i[0] for i in bottom_tied_tuples).items())
  top_tied_counts = list(Counter(i[0] for i in top_tied_tuples).items())
  bottom_true_counts = []

  for i in top_tied_counts:
    to_be_duplicated_count[i[0]] += round_half_up((t-len(to_be_duplicated))*(i[1]/temp_b))
  for j in bottom_tied_counts:
    bottom_true_counts.append((j[0], round_half_up((t-len(to_be_replaced))*(j[1]/temp_a))))

  grouped_bottom_tuples = defaultdict(list)
  for l in bottom_tied_tuples:
    grouped_bottom_tuples[l[0]].append(l)

  for n in bottom_true_counts:
    if n[0] in grouped_bottom_tuples:
      if len(grouped_bottom_tuples[n[0]]) > n[1]:
        for j in range(n[1]):
          to_be_replaced.append(grouped_bottom_tuples[n[0]][j])
      else:
        to_be_replaced.extend(grouped_bottom_tuples[n[0]])

  to_be_replaced = sorted(to_be_replaced, key=lambda x: x[1], reverse=True)
  #reverse the order so the obj of list_index can be deleted from the rear
  return to_be_replaced, to_be_duplicated_count

In [ ]:
class game():
  __slots__ = ['num_of_gamers', 'num_of_turns', 'TT', 'TB', 'BB', 'gamers_dict', 'dict_keys', 'clear_points', 'replacement_n', 'mistake_rate']
  def __init__(self):
    attributes_game = {'num_of_gamers': 0, 'num_of_turns': 10,
               'TT': [2,2], 'TB': [-1, 3], 'BB': [0, 0],
               'gamers_dict': {}, 'dict_keys': [], 'clear_points': 1,
               'replacement_n': 30, 'mistake_rate': 0}
    for name, value in attributes_game.items():
      setattr(self, name, value)

  def one_round(self, input1, input2):
    if input1 == input2 == 1: #(1, 1)
      return self.TT
    elif input1 == 1: #(1, 0)
      return self.TB
    elif input2 == 1: #(0, 1)
      return [self.TB[1], self.TB[0]]
    else: #(0, 0)
      return self.BB

  def add_gamer(self, gamer_type, n):
    if n <= 0:
      return
    if gamer_type not in self.dict_keys:
      self.dict_keys.append(type(gamer_type()))
      self.gamers_dict[type(gamer_type())] = []
    for _ in range(n):
      self.gamers_dict[type(gamer_type())].append(gamer_type())
    self.num_of_gamers += n
    return

  def remove_gamer_back(self, gamer_type, n):
    if n <= 0:
      return
    if type(gamer_type()) not in self.dict_keys:
      print('No such gamer')
      return
    if n > len(self.gamers_dict[type(gamer_type())]):
      print('Removal out of range')
      return
    del self.gamers_dict[type(gamer_type())][-n:]
    self.num_of_gamers -= n
    return

  def remove_gamer_index(self, gamer_type, n):
    if gamer_type not in self.dict_keys:
      print('No such gamer')
      return
    if n >= len(self.gamers_dict[gamer_type]) or n < -(len(self.gamers_dict[gamer_type])):
      print('Removal out of range')
      return
    del self.gamers_dict[gamer_type][n]
    self.num_of_gamers -= 1
    return

  def perturb(self, value): #mistake_rate_function
    if random.random() < self.mistake_rate:
      return 1 if value ==0 else 0
    return value

  def run_matches(self):
    #check if self.dict_keys is correct
    if self.dict_keys != list(self.gamers_dict.keys()):
      raise IncoherenceError('dict_keys isn\'t coherent with real keys')
    for list_x in self.gamers_dict.values():
      pairs = list(combinations(range(len(list_x)), 2))
      for pair in pairs:
        self.play_match(list_x[pair[0]], list_x[pair[1]])
    pairs = list(combinations(range(len(self.dict_keys)), 2))
    for pair in pairs:
      for gamer1 in self.gamers_dict[self.dict_keys[pair[0]]]:
        for gamer2 in self.gamers_dict[self.dict_keys[pair[1]]]:
          self.play_match(gamer1, gamer2)
    return

  def play_match(self, gamer1, gamer2):
    for i in range(self.num_of_turns):
      if self.mistake_rate == 0:
        outcome = self.one_round(gamer1.make_decision(), gamer2.make_decision())
      else:
        gamer1.output = self.perturb(gamer1.make_decision())
        gamer2.output = self.perturb(gamer2.make_decision())
        outcome = self.one_round(gamer1.output, gamer2.output)
      gamer1.points += outcome[0]
      gamer2.points += outcome[1]
      gamer1.former_round = gamer2.output
      gamer2.former_round = gamer1.output
    gamer1.end_of_turns()
    gamer2.end_of_turns()

  def print_points(self, n):
    for key in self.gamers_dict:
      print(f"{key}[{n}] value: {self.gamers_dict[key][n].points}")

  def print_points_extreme(self, max_or_min):
    if max_or_min:
      for key in self.gamers_dict:
        print(key, 'max_value:', max([i.points for i in self.gamers_dict[key]]))
    else:
      for key in self.gamers_dict:
        print(key, 'max_value:', min([i.points for i in self.gamers_dict[key]]))
    return

  def print_points_avg(self):
    for key in self.gamers_dict:
      print(key, 'avg: ', round(sum([i.points for i in self.gamers_dict[key]])/len(self.gamers_dict[key])))

  def print_points_stdev(self):
    for key in self.gamers_dict:
      print(key, ' stdev: ', np.std([i.points for i in self.gamers_dict[key]]))

  def print_gamer_nums(self):
    for key in self.gamers_dict:
      print(key, ':', len(self.gamers_dict[key]))

  def replacement_process(self):
    gamer_tuples = []
    for listx in self.gamers_dict.values():
      for i in range(len(listx)):
        gamer_tuples.append((type(listx[i]), i, listx[i].points))
        if self.clear_points:
          listx[i].points = 0
    to_be_replaced, to_be_duplicated = get_replacement(gamer_tuples, self.replacement_n)
    for i in to_be_replaced:
      self.remove_gamer_index(i[0], i[1])
    for key, value in to_be_duplicated.items():
      self.add_gamer(key, value)
    for key in list(self.gamers_dict.keys()):
      if self.gamers_dict[key] == []:
        del self.gamers_dict[key]
    self.dict_keys = list(self.gamers_dict.keys())
    return

  def complete_round(self):
    self.run_matches()
    self.replacement_process()
    return

In [ ]:
def new_test_case():
  game1 = game()
  game1.num_of_turns = 10
  game1.mistake_rate = 0.05
  game1.add_gamer(copycat, 100)
  game1.add_gamer(the_believer, 100)
  game1.add_gamer(the_selfish, 100)
  game1.add_gamer(pavlov, 100)
  game1.add_gamer(the_stubborn, 100)
  game1.add_gamer(the_generous_copycat, 100)
  game1.add_gamer(majority_follower, 100)
  game1.add_gamer(the_tricker, 100)
  game1.add_gamer(the_mean_copycat, 100)
  game1.add_gamer(detective, 100)
  tmp_round = 10
  for i in range(tmp_round):
    if i == tmp_round-1:
      game1.clear_points = 0
    game1.complete_round()
  print(game1.num_of_gamers)
  game1.print_gamer_nums()
  print('\n')
  game1.print_points(0)
  print('\n')
  game1.print_points_stdev()
  print('\n')
  game1.print_points_avg()

In [ ]:
import time
start_time = time.time()
new_test_case()
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

1001
<class '__main__.copycat'> : 116
<class '__main__.the_believer'> : 100
<class '__main__.pavlov'> : 100
<class '__main__.the_stubborn'> : 123
<class '__main__.the_generous_copycat'> : 302
<class '__main__.majority_follower'> : 158
<class '__main__.the_mean_copycat'> : 102


<class '__main__.copycat'>[0] value: 16504
<class '__main__.the_believer'>[0] value: 16976
<class '__main__.pavlov'>[0] value: 16378
<class '__main__.the_stubborn'>[0] value: 16339
<class '__main__.the_generous_copycat'>[0] value: 17076
<class '__main__.majority_follower'>[0] value: 16729
<class '__main__.the_mean_copycat'>[0] value: 16248


<class '__main__.copycat'>  stdev:  124.85222269503497
<class '__main__.the_believer'>  stdev:  146.39333693853695
<class '__main__.pavlov'>  stdev:  132.4677908021418
<class '__main__.the_stubborn'>  stdev:  163.75200098530235
<class '__main__.the_generous_copycat'>  stdev:  4682.743009154755
<class '__main__.majority_follower'>  stdev:  2959.901059150989
<class '__main__.t

In [ ]:
k = game()
k.add_gamer(the_mean_copycat, 1)
k.add_gamer(copycat, 1)
k.run_matches()
k.print_points(0)

<class '__main__.the_mean_copycat'>[0] value: 20
<class '__main__.copycat'>[0] value: 20


In [ ]:
cProfile.run('new_test_case()')

1000
<class '__main__.copycat'> : 116
<class '__main__.the_believer'> : 100
<class '__main__.pavlov'> : 100
<class '__main__.the_stubborn'> : 116
<class '__main__.the_generous_copycat'> : 280
<class '__main__.majority_follower'> : 188
<class '__main__.the_mean_copycat'> : 100


<class '__main__.copycat'>[0] value: 16655
<class '__main__.the_believer'>[0] value: 16758
<class '__main__.pavlov'>[0] value: 16479
<class '__main__.the_stubborn'>[0] value: 16244
<class '__main__.the_generous_copycat'>[0] value: 17000
<class '__main__.majority_follower'>[0] value: 17030
<class '__main__.the_mean_copycat'>[0] value: 16401


<class '__main__.copycat'>  stdev:  152.40013079466394
<class '__main__.the_believer'>  stdev:  129.5383383404311
<class '__main__.pavlov'>  stdev:  153.77006990958935
<class '__main__.the_stubborn'>  stdev:  152.74349578738457
<class '__main__.the_generous_copycat'>  stdev:  4765.49295329942
<class '__main__.majority_follower'>  stdev:  2974.8537488540633
<class '__main__.t